In [1]:
import pandas as pd
import json




In [2]:
#Calcolo delle grandezze utilizzate nei test di efficacia
ORIGINAL_SIZE=568454            #dimensione originale
reduction_factor=[0.1,0.3,0.5,0.7,1,1.3,1.5,1.7,2]  #fattori di riduzione utilizzati

NEW_SIZE=[int(ORIGINAL_SIZE * x) for x in reduction_factor]


In [3]:
#path dei raw data
path_MR="MR_raw_data.txt"
path_HIVE="HIVE_Raw_data.txt"
path_SPARK_CORE="SPARK_CORE_Raw_data.txt"
path_SPARK_SQL="SPARK_SQL_Raw_Data.txt"

In [4]:
def extract_data(file):
    tempi_per_job={'job1':[],'job2':[]}
    current_job=''
    with open(file,'r') as f:
        lines=f.readlines()
        for line in lines:
            value=line.strip().replace('\n','').lower()
            if 'job' in value:
                value=value.replace(' ','')
                current_job=value
            elif 'real' in value:
                value=value.strip().replace('\n','').split('\t')
                if len(value)>1:
                    time=value[1]
                    tempi_per_job[current_job].append(time)
    return tempi_per_job



            
    


In [5]:
#acquisizione dei dati
dati_SPARK_CORE=extract_data(path_SPARK_CORE)
dati_SPARK_SQL=extract_data(path_SPARK_SQL)
dati_HIVE=extract_data(path_HIVE)
dati_MR=extract_data(path_MR)
dati_SPARK_SQL



{'job1': ['0m28,374s',
  '0m41,220s',
  '0m45,507s',
  '0m53,927s',
  '1m0,502s',
  '1m6,997s',
  '1m12,927s',
  '1m19,662s',
  '1m26,289s'],
 'job2': ['0m8,670s',
  '0m8,763s',
  '0m8,605s',
  '0m8,744s',
  '0m8,992s',
  '0m9,059s',
  '0m9,632s',
  '0m9,723s',
  '0m9,838s']}

In [6]:
#creazione della mappatura k->dimensione attuale
def create_mapping(keys,values,tgt_path):
    mapping=dict()
    for i in range(0,len(keys)):
        mapping[keys[i]]=values[i]
        
    # covnersione mappa in json
    json_data = json.dumps(mapping)

    # Save the JSON data to a file
    with open(tgt_path, "w") as file:
        file.write(json_data)

In [7]:
#creazione di una viste  (colonne sono il fattore riduttivo) e di un mapping fattore_riduzione->dimensione attuale

columns_view2=reduction_factor
print(columns_view2)
path_json_map='mapping_k_dim.json'
create_mapping(columns_view2,NEW_SIZE,path_json_map)



[0.1, 0.3, 0.5, 0.7, 1, 1.3, 1.5, 1.7, 2]


In [8]:
def format_values(values):
    out=[]
        # Remove the 'm' and 's' characters and replace the comma with a dot
    for v in values:
        time_string = v.replace("m", "").replace("s", "").replace(",", ".")

        # Convert the string to a float
        v = float(time_string)
        print("Tempo corrente:",v)
        out.append(v)

    return out

def gen_rows(tempi,sistema):
    row1=[sistema]
    row2=[sistema]
    print("Sistema",sistema)
    for k in tempi.keys():
        print("Job:",k)
        if k=='job1':
            row1.append('1')
            values=tempi[k]
            values=format_values(values)
            row1.extend(values)
            
        elif k=='job2':
            row2.append('2')
            values=tempi[k]
            values=format_values(values)
            row2.extend(values)
        
    return [row1,row2]
        
def create_dataset(path,columns,data,systems):
    out_put=[]
    for i in range(0,len(data)):
        tempi=data[i]
        sistema=systems[i]
        rows=gen_rows(tempi,sistema)
        out_put=out_put+rows
        print('dati correnti',out_put)
    
    pd.DataFrame(data=out_put,columns=columns).to_csv(path,index=False)




In [9]:


columns=['System','job']+columns_view2
print("NUMERO COLONNE:",len(columns))
data=[dati_SPARK_CORE,dati_HIVE,dati_MR,dati_SPARK_SQL]
systems=['SPARK_CORE','HIVE','MAP-REDUCE','SPARK-SQL']

create_dataset('tempi.csv',columns,data,systems)

NUMERO COLONNE: 11
Sistema SPARK_CORE
Job: job1
Tempo corrente: 9.583
Tempo corrente: 11.824
Tempo corrente: 16.012
Tempo corrente: 18.426
Tempo corrente: 21.965
Tempo corrente: 25.213
Tempo corrente: 27.975
Tempo corrente: 29.687
Tempo corrente: 33.084
Job: job2
Tempo corrente: 7.117
Tempo corrente: 7.854
Tempo corrente: 10.562
Tempo corrente: 11.514
Tempo corrente: 12.986
Tempo corrente: 14.367
Tempo corrente: 15.415
Tempo corrente: 16.591
Tempo corrente: 18.521
dati correnti [['SPARK_CORE', '1', 9.583, 11.824, 16.012, 18.426, 21.965, 25.213, 27.975, 29.687, 33.084], ['SPARK_CORE', '2', 7.117, 7.854, 10.562, 11.514, 12.986, 14.367, 15.415, 16.591, 18.521]]
Sistema HIVE
Job: job1
Tempo corrente: 40.496
Tempo corrente: 55.58
Tempo corrente: 15.187
Tempo corrente: 119.261
Tempo corrente: 135.07
Tempo corrente: 152.103
Tempo corrente: 159.95
Tempo corrente: 212.162
Tempo corrente: 220.862
Job: job2
Tempo corrente: 40.339
Tempo corrente: 40.729
Tempo corrente: 43.746
Tempo corrente: 45.95